In [ ]:
# !pip install pyreadr
# !pip install plotly

# !conda install -c conda-forge nodejs -y
# !conda install -c conda-forge/label/gcc7 nodejs -y
# !conda install -c conda-forge/label/cf201901 nodejs -y
# !conda install -c conda-forge/label/cf202003 nodejs -y

# !jupyter labextension install jupyterlab-plotly
# !pip install scipy
# !pip install -U kaleido
# !pip install networkx
# !pip install matplotlib
# !pip install igraph

In [ ]:
import pyreadr
import os
import pandas as pd
import numpy as np
import plotly as plt
import plotly.express as px
import scipy
import plotly.figure_factory as ff
import kaleido
import matplotlib.pyplot as plt

from matplotlib.pyplot import figure


from pair_matching import discrepancyMatrix

import igraph as ig
from igraph import Graph

In [ ]:
ag_data = pd.read_csv("agdata_smoke.csv", sep=",", low_memory=False)
ag_data.shape

W - exposure to smoking variable

In [ ]:
smokers = ag_data[ag_data["W"] == 0] # smoker
non_smokers = ag_data[ag_data["W"] == 1] # non-smoker


print("Number of smokers - {0}".format(len(smokers)))
print("Number of non-smokers - {0}".format(len(non_smokers)))

In [ ]:
df = ag_data.copy()

df["W_str"] = df["W"].map({1: "No", 0: 'Yes'})

In [ ]:
### Sex

fig = px.histogram(df, x="W_str", color="sex", barnorm='percent', text_auto=".2f",
                   width=800, height=400)

fig.update_layout(
    title_text='AG Project: Ratio between male and female (non)smokers', # title of plot
    xaxis_title_text='Smoking', # xaxis label
    yaxis_title_text='Percentage', # yaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.show()

#create plots dir
if not os.path.exists("plots"):
    os.mkdir("plots")

# fig.write_image("plots/AG_sex.png")

In [ ]:
### Age

fig = px.histogram(df, x="W_str", color="age_cat", barnorm='percent', text_auto=".2f",
                   width=800, height=400)

fig.update_layout(
    title_text='AG Project: Age ratio between (non)smokers', # title of plot
    xaxis_title_text='Smoking', # xaxis label
    yaxis_title_text='Percentage', # yaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.show()
# fig.write_image("plots/AG_age.png")

In [ ]:
### BMI

hist_data = [smokers["bmi_corrected"], non_smokers["bmi_corrected"]]

group_labels = ['Smokers', 'Non-smokers']

colors = ['slategray', 'magenta']

fig = ff.create_distplot(hist_data, group_labels, bin_size=2, show_rug=False,
                         histnorm="probability density", colors=colors)


fig.update_layout(
    title_text='AG Project: BMI probability density of (non)smokers', # title of plot
    xaxis_title_text="BMI (kg/m2)", # xaxis label
    yaxis_title_text='Probability density', # yaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.01 # gap between bars of the same location coordinates
)
fig.show()
# fig.write_image("plots/AG_bmi.png")

## Create pairs of samples

In [ ]:
df_match = ag_data.copy()
df_match.shape

In [ ]:
df_match["is_treated"] = df_match["W"].astype(bool)
df_match["pair_nb"] = np.nan

df_match.shape

In [ ]:
# Optional weights for each covariate when computing the distances
# WARNING: the order of the items in scaling needs to be the same as the order of the covariates (i.e. columns)
scaling =  np.ones((df_match.shape[1],), dtype=int)

scaling.shape

In [ ]:
sex_ix = df_match.columns.get_loc("sex")
age_cat_ix = df_match.columns.get_loc("age_cat")
bmi_corrected_ix = df_match.columns.get_loc("bmi_corrected")

In [ ]:
# set the thresholds for each covariate, default is Inf (i.e. no matching)
thresholds =  np.empty((df_match.shape[1], ))
thresholds[:] = np.nan

# set particular values
thresholds[sex_ix] = 0
thresholds[age_cat_ix] = 1
thresholds[bmi_corrected_ix] = 4

In [ ]:
relevant_fields = ["sex", "age_cat", "bmi_corrected", "is_treated"]

In [ ]:
# ask Stephan why we are not reindexing
# treated_units = df_match[df_match["is_treated"] == True].reset_index(drop=True)
# control_units = df_match[df_match["is_treated"] == False].reset_index(drop=True)

treated_units = df_match[df_match["is_treated"] == True]
control_units = df_match[df_match["is_treated"] == False]

N_treated = treated_units.shape[0]
N_control = control_units.shape[0]

print("Number of treated units: {0}".format(N_treated))
print("Number of control units: {0}".format(N_control))

In [ ]:
discrepancies = discrepancyMatrix(treated_units, control_units, thresholds, scaling)
discrepancies.shape

### Network construction

In [ ]:
# if there is ni edge -> True
adj = np.isnan(discrepancies)

#indices of existing edges
edges_mat = np.argwhere(adj == False)

adj.shape, edges_mat.shape

!!! indices are sorted

In [ ]:
weights = []

for i in range(0, edges_mat.shape[0]):
    
    row = edges_mat[i][0]
    col = edges_mat[i][1]
    
    w = 1 / (1 + discrepancies[row][col])
    
    weights.append(w)
    
weights = np.array(weights)
weights.shape

In [ ]:
edges_vector = edges_mat

for i in range(0, edges_vector.shape[0]):
    edges_mat[i][1] = edges_vector[i][1] + N_treated

In [ ]:
edges_vector = edges_mat.flatten()
edges_vector.shape

In [ ]:
t_nodes = np.repeat(True, N_treated)
c_nodes = np.repeat(False, N_control)

nodes = np.concatenate((t_nodes, c_nodes), axis=None)
nodes.shape

In [ ]:
g = ig.Graph.Bipartite(nodes, edges_mat)
assert g.is_bipartite()

In [ ]:
matching = g.maximum_bipartite_matching(weights=weights)

In [ ]:
print("Number of pairs - {0}".format(len(g.es)))

In [ ]:
pairs_dict = dict()
N_matched = 0

for i in range(0, N_treated):
    if matching.is_matched(i):
        N_matched += 1
        pairs_dict[N_matched] = [i, matching.match_of(i) - N_treated]

In [ ]:
pairs_dict[1][0]

In [ ]:
treated_units["pair_nb"][pairs_dict[1][0]]

In [ ]:
#sanity check
matched = []
total_nb_match = 0

for i in range(1, N_matched):
    total_nb_match = total_nb_match + 1
    
    # save pair number
    treated_units["pair_nb"][pairs_dict[i][0]] = total_nb_match
    control_units["pair_nb"][pairs_dict[i][1]] = total_nb_match
    
    matched.append(treated_units.iloc[pairs_dict[i][0], :])
    matched.append(control_units.iloc[pairs_dict[i][1], :])

In [ ]:
matched_df = pd.DataFrame(matched, columns=treated_units.columns)

In [ ]:
print("Number of pairs: {0}".format(len(matched_df.W)))
print("Number of smokers: {0}".format(len(matched_df[matched_df.W == 0])))
print("Number of non-smokers: {0}".format(len(matched_df[matched_df.W == 1])))

### Plots

In [ ]:
df1 = matched_df.copy()

df1["W_str"] = df1["W"].map({1: "No", 0: 'Yes'})

In [ ]:
### Sex

fig = px.histogram(df1, x="W_str", color="sex", barnorm='percent', text_auto=".2f",
                   width=800, height=400)

fig.update_layout(
    title_text='AG Project: Ratio between male and female (non)smokers', # title of plot
    xaxis_title_text='Smoking', # xaxis label
    yaxis_title_text='Percentage', # yaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.show()

#create plots dir
if not os.path.exists("plots"):
    os.mkdir("plots")
    
fig.write_image("plots/AG_sex_after_matching.png.png")

In [ ]:
### Age
fig = px.histogram(df1, x="W_str", color="age_cat", barnorm='percent', text_auto=".2f",
                   width=800, height=400)

fig.update_layout(
    title_text='AG Project: Age ratio between (non)smokers', # title of plot
    xaxis_title_text='Smoking', # xaxis label
    yaxis_title_text='Percentage', # yaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.show()
fig.write_image("plots/AG_age_after_matching.png")

In [ ]:
### BMI

smokers1 = df1[df1["W"] == 0] # smoker
non_smokers1 = df1[df1["W"] == 1] # non-smoker

hist_data1 = [smokers1["bmi_corrected"], non_smokers1["bmi_corrected"]]

group_labels = ['Smokers', 'Non-smokers']

colors = ['slategray', 'magenta']

fig = ff.create_distplot(hist_data1, group_labels, bin_size=2, show_rug=False,
                         histnorm="probability density", colors=colors)


fig.update_layout(
    title_text='AG Project: BMI probability density of (non)smokers', # title of plot
    xaxis_title_text="BMI (kg/m2)", # xaxis label
    yaxis_title_text='Probability density', # yaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.01 # gap between bars of the same location coordinates
)
fig.show()
fig.write_image("plots/AG_bmi_after_matching.png")

### Sampling

In [ ]:
# define number of observations, number of treatment and number of 

# randomizations (n_col)
n_total = matched_df.W.shape[0]
n_treated = matched_df.W.sum()
n_col = 10**5

# init empty matrix
W_sim = np.empty([n_total, n_col])

for t in range(n_col):
    W_sim_to_fill = np.empty(n_total)
    flip_coin = np.random.binomial(n = 1, p = 0.5, size = n_treated)
    W_sim_to_fill[np.arange(start = 0, stop = (n_total - 1), step = 2)] = flip_coin
    W_sim_to_fill[np.arange(start = 1, stop = n_total, step = 2)] = 1 - flip_coin
    W_sim[:, t] = W_sim_to_fill

In [ ]:
print("Number of duplicated columns: {0}".format(pd.DataFrame(W_sim).columns.duplicated().sum()))